# Import packages

In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from common import *
from sklearn.preprocessing import StandardScaler

# 1. Load dataset

In [2]:
train_data = pd.read_csv(join(PATH.input, "train_data.csv"))
sample_submission = pd.read_csv(join(PATH.input, "sample_submission.csv"))
train_data = train_data.fillna(method = 'bfill')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3859200 entries, 0 to 3859199
Data columns (total 13 columns):
 #   Column   Dtype  
---  ------   -----  
 0   TurbID   int64  
 1   Day      int64  
 2   Tmstamp  object 
 3   Wspd     float64
 4   Wdir     float64
 5   Etmp     float64
 6   Itmp     float64
 7   Ndir     float64
 8   Pab1     float64
 9   Pab2     float64
 10  Pab3     float64
 11  Prtv     float64
 12  Patv     float64
dtypes: float64(10), int64(2), object(1)
memory usage: 382.8+ MB


# 2. Preprocessing

In [3]:
tms_list = list(pd.unique(train_data['Tmstamp']))
train_data['Tmstamp'] = train_data['Tmstamp'].apply(lambda x: tms_list.index(x)+1)
train_x, train_y = make_train_data(train_data, 5, 200)

100%|██████████| 134/134 [00:21<00:00,  6.24it/s]


In [4]:
train_x = np.array(train_x).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])
train_y = np.array(train_y)
train_x.shape, train_y.shape

((25996, 720, 11), (25996, 288))

In [5]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x.reshape(-1, train_x.shape[-1])).reshape(train_x.shape)
train_x.shape, train_y.shape

((25996, 720, 11), (25996, 288))

# 3. Modeling

In [6]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

tf.keras.utils.set_random_seed(RANDOM_STATE)
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

2022-07-11 05:46:43.898688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 05:46:43.905958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 05:46:43.906269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [7]:
model = Sequential([
    GRU(256, input_shape=train_x[0].shape),
    Dense(516, activation='relu'),
    Dense(288, activation='relu')
])
optimizer = tf.optimizers.RMSprop(0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

2022-07-11 05:46:43.994465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-11 05:46:43.995408: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 05:46:43.995770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 05:46:43.996031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

# 4. Training

In [8]:
model.fit(train_x, train_y, epochs=10, batch_size=128);

2022-07-11 05:46:44.444217: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 823553280 exceeds 10% of free system memory.
2022-07-11 05:46:44.815478: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 823553280 exceeds 10% of free system memory.


Epoch 1/10


2022-07-11 05:46:46.530020: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  3/204 [..............................] - ETA: 7s - loss: 295104.1562 - mae: 345.8313  

2022-07-11 05:46:47.058088: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


204/204 [==============================] - 10s 37ms/step - loss: 192472.8438 - mae: 334.2208
Epoch 2/10
204/204 [==============================] - 8s 37ms/step - loss: 170263.3438 - mae: 323.0628
Epoch 3/10
204/204 [==============================] - 7s 37ms/step - loss: 161118.0469 - mae: 313.3703
Epoch 4/10
204/204 [==============================] - 7s 37ms/step - loss: 152653.8594 - mae: 303.2585
Epoch 5/10
204/204 [==============================] - 8s 37ms/step - loss: 141967.1875 - mae: 289.6254
Epoch 6/10
204/204 [==============================] - 8s 37ms/step - loss: 132136.2500 - mae: 276.6620
Epoch 7/10
204/204 [==============================] - 8s 37ms/step - loss: 123856.8750 - mae: 265.6998
Epoch 8/10
204/204 [==============================] - 7s 37ms/step - loss: 116824.2422 - mae: 257.0731
Epoch 9/10
204/204 [==============================] - 8s 37ms/step - loss: 110378.8438 - mae: 248.4108
Epoch 10/10
204/204 [==============================] - 8s 37ms/step - loss: 104095.

# 5. Predict

In [9]:
test_data_list = [x for x in range(196, 201)]
test_data      = train_data[train_data["Day"].isin(test_data_list)]
test_data      = test_data.drop(["TurbID", "Day"], axis=1)
test_data      = np.array(test_data).reshape(-1, train_x[0].shape[0], train_x[0].shape[1])
test_data.shape

(134, 720, 11)

In [10]:
test_data = scaler.transform(test_data.reshape(-1, train_x.shape[-1])).reshape(test_data.shape)
test_data.shape

(134, 720, 11)

# 6. Generate submission

In [11]:
sample_submission['Patv'] = model.predict(test_data).reshape(-1)
sample_submission.to_csv(join(PATH.output, "proposed1.csv"), index=False)

5/5 [==============================] - 0s 12ms/step
